In [1]:
%pip install torch
%pip install transformers[torch]
%pip install ipywidgets
%pip install datasets
%pip install -U scikit-learn
%pip install -U ipywidgets
%pip install 'accelerate>=1.1.0'

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch
import transformers
import torch.nn as nn
import pandas as pd
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm
from transformers import Trainer, TrainingArguments
from datasets import Dataset
from itertools import product
from collections import defaultdict

pd.set_option('display.width',1000)


In [ ]:
RANDOM_SEED = 42
torch.manual_seed(RANDOM_SEED)

EPOCHS = 3
MAX_LENGTH = 128
K = 5


device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cuda')

In [103]:
from load_data import DontPatronizeMe

DATA_DIR = 'data/'
TEST_PATH = f'{DATA_DIR}task4_test.tsv'

dpm = DontPatronizeMe(DATA_DIR, TEST_PATH)
dpm.load_task1()

data = dpm.train_task1_df

trids = pd.read_csv(f'{DATA_DIR}train_semeval_parids-labels.csv')
teids = pd.read_csv(f'{DATA_DIR}dev_semeval_parids-labels.csv')

trids['par_id'] = trids.par_id.astype(str)
teids['par_id'] = teids.par_id.astype(str)

cols = ['par_id', 'text', 'label_y']

trdf = trids.merge(data, on='par_id', how='left')[cols]
tedf = teids.merge(data, on='par_id', how='left')[cols]

# rename label_y to label for Trainer
trdf.rename(columns={"label_y": "labels"}, inplace=True)
tedf.rename(columns={"label_y": "labels"}, inplace=True)

print(trdf.head())
print()
print(tedf.head())
print()

trds = Dataset.from_pandas(trdf)
print(trds)


  par_id                                               text  labels
0   4341  The scheme saw an estimated 150,000 children f...       1
1   4136  Durban 's homeless communities reconciliation ...       1
2  10352  The next immediate problem that cropped up was...       1
3   8279  Far more important than the implications for t...       1
4   1164  To strengthen child-sensitive social protectio...       1

  par_id                                               text  labels
0   4046  We also know that they can benefit by receivin...       1
1   1279  Pope Francis washed and kissed the feet of Mus...       1
2   8330  Many refugees do n't want to be resettled anyw...       1
3   4063  "Budding chefs , like "" Fred "" , "" Winston ...       1
4   4089  "In a 90-degree view of his constituency , one...       1

Dataset({
    features: ['par_id', 'text', 'labels'],
    num_rows: 8375
})


In [104]:
counts = trdf['labels'].value_counts()
print("Original class distribution:")
print(counts)
print()

weights = torch.tensor([counts[0]/counts[0], counts[0]/counts[1]], dtype=torch.float).to(device)
print("Class weights:", weights)
print()

# # Upsample positive class
# pos_df = trdf[trdf['label_y'] == 1]
# neg_df = trdf[trdf['label_y'] == 0]

# upsampled_pos_df = pos_df.sample(n=len(neg_df), replace=True, random_state=RANDOM_SEED)
# trdf_balanced = pd.concat([neg_df, upsampled_pos_df]).sample(frac=1, random_state=RANDOM_SEED).reset_index(drop=True)

# print("\nBalanced class distribution:")
# print(trdf_balanced['label_y'].value_counts())

Original class distribution:
labels
0    7581
1     794
Name: count, dtype: int64

Class weights: tensor([1.0000, 9.5479], device='cuda:0')



In [105]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=K, shuffle=True, random_state=RANDOM_SEED)

X = trdf["text"].values
y = trdf["labels"].values

folds = list(skf.split(X, y))


In [106]:
from sklearn.metrics import f1_score

class BalancedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.pop("labels").view(-1)

        outputs = model(**inputs)
        logits = outputs.logits.view(-1, 2)

        loss_fn = nn.CrossEntropyLoss(weight=weights)
        loss = loss_fn(logits, labels)

        return (loss, outputs) if return_outputs else loss

def compute_metrics(p):
    preds = p.predictions.argmax(-1)
    labels = p.label_ids
    f1 = f1_score(labels, preds, pos_label=1)
    return { "f1": f1 }

In [107]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification


rob_checkpoint = "FacebookAI/roberta-base"
deb_checkpoint = "microsoft/deberta-base"

rob_tokenizer = AutoTokenizer.from_pretrained(rob_checkpoint)
deb_tokenizer = AutoTokenizer.from_pretrained(deb_checkpoint)

def tokenize(tokenizer, batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=MAX_LENGTH)

architectures = [("Roberta", rob_checkpoint, rob_tokenizer), ("Deberta", deb_checkpoint, deb_tokenizer)]


In [ ]:
LRS = [2e-5, 3e-5]
BSZS = [16, 32]

num_epochs = len(folds) * len(architectures) * len(LRS) * len(BSZS) * EPOCHS
print(f"Total training epochs: {num_epochs}")
print()

# Determine best hyperparameters using k-fold cross-validation
def hyperparameter_search():
    hp_metrics = {}
    hp_metrics["Roberta"] = defaultdict(float)
    hp_metrics["Deberta"] = defaultdict(float)

    for name, checkpoint, tokenizer in architectures:
        print(f"Training {name}...")
        print()

        for fold, (train_idx, val_idx) in enumerate(folds):
            print(f"Fold {fold + 1}/{K}")

            trds = Dataset.from_pandas(trdf.iloc[train_idx])
            valds = Dataset.from_pandas(trdf.iloc[val_idx])

            trds = trds.map(lambda batch: tokenize(tokenizer, batch), batched=True)
            valds = valds.map(lambda batch: tokenize(tokenizer, batch), batched=True)

            trds.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
            valds.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

            for lr, bsz in product(LRS, BSZS):
                model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2).to(device)
                training_args = TrainingArguments(
                    learning_rate=lr,
                    per_device_train_batch_size=bsz,
                    per_device_eval_batch_size=bsz,
                    num_train_epochs=EPOCHS,
                    logging_strategy="epoch",
                    eval_strategy="epoch",
                    # save_strategy="epoch",
                    # save_total_limit=1,
                    # load_best_model_at_end=True,
                    # metric_for_best_model="f1",
                    remove_unused_columns=False,
                )

                trainer = BalancedTrainer(
                    model=model,
                    args=training_args,
                    train_dataset=trds,
                    eval_dataset=valds,
                    compute_metrics=compute_metrics
                )

                trainer.train()
                metrics = trainer.evaluate()

                hp_metrics[name][(lr, bsz)] += metrics["eval_f1"]
            
    res = {}
    for name in hp_metrics:
        best_hp = max(hp_metrics[name], key=hp_metrics[name].get)
        res[name] = {
            "best_hps": best_hp,
            "avg_f1": hp_metrics[name][best_hp] / K
        }

    return res

res = hyperparameter_search()
print(res)

Total training epochs: 120

Training Roberta...

Fold 1/5


Map:   0%|          | 0/6700 [00:00<?, ? examples/s]

Map:   0%|          | 0/1675 [00:00<?, ? examples/s]

Loading weights:   0%|          | 0/197 [00:00<?, ?it/s]

RobertaForSequenceClassification LOAD REPORT from: FacebookAI/roberta-base
Key                             | Status     | 
--------------------------------+------------+-
lm_head.layer_norm.weight       | UNEXPECTED | 
roberta.embeddings.position_ids | UNEXPECTED | 
lm_head.dense.bias              | UNEXPECTED | 
lm_head.layer_norm.bias         | UNEXPECTED | 
lm_head.dense.weight            | UNEXPECTED | 
lm_head.bias                    | UNEXPECTED | 
classifier.dense.bias           | MISSING    | 
classifier.out_proj.bias        | MISSING    | 
classifier.out_proj.weight      | MISSING    | 
classifier.dense.weight         | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.


Epoch,Training Loss,Validation Loss


In [ ]:
# Retrain best models on full training set
def train_models(best_hyperparams):
    models = {}

    for name, checkpoint, tokenizer in architectures:
        print(f"Retraining {name} on full training set...")

        trds = Dataset.from_pandas(trdf)
        valds = Dataset.from_pandas(tedf)

        trds = trds.map(lambda batch: tokenize(tokenizer, batch), batched=True)
        valds = valds.map(lambda batch: tokenize(tokenizer, batch), batched=True)

        trds.set_format(type="torch", columns=["input_ids", "attention_mask", "label_y"])
        valds.set_format(type="torch", columns=["input_ids", "attention_mask", "label_y"])

        lr, bsz = best_hyperparams[name]

        model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2).to(device)
        training_args = TrainingArguments(
            output_dir=f'./final_results/{name}',
            learning_rate=lr,
            per_device_train_batch_size=bsz,
            per_device_eval_batch_size=bsz,
            num_train_epochs=EPOCHS,
            logging_dir=f'./final_logs/{name}',
            save_strategy="epoch"
        )

        trainer = BalancedTrainer(
            model=model,
            args=training_args,
            train_dataset=trds,
            eval_dataset=valds
        )

        trainer.train()
        metrics = trainer.evaluate()

        models[name] = (model, tokenizer, metrics["eval_f1"])
    
    # Calculate relative F1 scores for ensembling
    tot_f1 = sum(m[2] for m in models.values())
    for name in models:
        model, tokenizer, f1 = models[name]

        models[name] = (model, tokenizer, f1 / tot_f1)

    return models


In [ ]:
# Helper function to save predictions to an output file
def labels2file(p, outf_path):
    pass

In [ ]:
# TODO: may be possible to just use trainer output dir
# Predict for dev set and save to os.path.join('res/', 'dev.txt')
# Save gold labels for dev set to os.path.join('ref/', 'dev.txt')

In [ ]:
!python3 evaluation.py . .
!cat scores.txt

In [ ]:
# dpm.load_test()
# Predict for test set and save to os.path.join('res/', 'test.txt')